In [1]:
import pandas as pd
import numpy as np
import duckdb

In [2]:
# example setting the sample size to 100000
duckdb.default_connection.execute("SET GLOBAL pandas_analyze_sample=100000")


## Loading GL AND PR data 

In [3]:
gl_file_excel = pd.read_csv('gl.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_13224\1320238739.py:1: DtypeWarning: Columns (6,7,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  gl_file_excel = pd.read_csv('gl.csv')


In [4]:
gl_file_excel.shape

(92046, 19)

In [5]:
pr_file_excel = pd.read_csv("pr.csv")

In [6]:
pr_file_excel.shape

(89605, 63)

In [7]:
gl_file_excel.head(1)

,MY GSTIN,Type,Supplier Name,Supplier GSTIN/UIN,Invoice No,Invoice Date,Original Invoice No,Original Invoice date,Total Invoice value,Place of Supply,Total Taxable Amount,Integrated Tax Amount,Central Tax Amount,State Tax Amount,Cess Amount,ITC Available - Cess,Remarks,HL Remarks,Final Remarks
0,29AADCH3844D1ZO,B2B,SOLUTIONS INFINI TECHNOLOGIES (I) Pvt Ltd,29AANCS0353C1ZF,NaN,NaN,NaN,NaN,2001.55,Karnataka,2001.55,13.98,173.15,173.15,NaN,NaN,Supplier GSTIN missing+Supplier Tax Invoice No...,Ignore No GST ( Journal Entry),NaN


In [8]:
# # Convert all columns to strings
# gl_file_excel = gl_file_excel.astype(str)
# pr_file_excel = pr_file_excel.astype(str)

In [9]:
# Convert to datetime
gl_file_excel['Invoice Date'] = pd.to_datetime(gl_file_excel['Invoice Date'], format='%d-%m-%Y', errors='coerce')

# Format as 'dd-mm-yyyy'
gl_file_excel['Invoice Date'] = gl_file_excel['Invoice Date'].dt.strftime('%d-%m-%Y')

In [10]:
# Convert to datetime
pr_file_excel['Document Date (PR)'] = pd.to_datetime(pr_file_excel['Document Date (PR)'], format='%d-%m-%Y', errors='coerce')

# Format as 'dd-mm-yyyy'
pr_file_excel['Document Date (PR)'] = pr_file_excel['Document Date (PR)'].dt.strftime('%d-%m-%Y')

In [11]:
pr_file_excel.head(1)

,ITC Claim month,Matching Key,Match Status,Match Status Description,Matching at Pan/ GSTIN,Sub-Buckets,Mismatched Fields,Mismatched Fields Count,Supplier Name,Supplier GSTIN (PR),...,Financial Year,Voucher Number,Out of Range (2B),Out of Range (PR),Claimable ITC - CGST,Claimable ITC - SGST,Claimable ITC - IGST,Claimable ITC - Cess,CT Remarks,Deferred ITC Month
0,2024 01,f5552d7f441c141c,Exact,NaN,My Gstin and Supplier Gstin Level,Actioned with No Changes,Taxable Value,1,TRINITY PANEL,29AAMFT6309F1ZV,...,2023-24,Jan'24,False,False,1737.9,1737.9,0.0,0,NaN,NaN


In [12]:
gl_file = gl_file_excel.copy()

In [13]:
pr_file = pr_file_excel.copy()

In [14]:
gl_file.shape

(92046, 19)

In [15]:
pr_file.shape

(89605, 63)

In [16]:
gl_1 = duckdb.sql("""
              select 
                    "Supplier GSTIN/UIN",
                    "MY GSTIN",
                    regexp_replace("Invoice No", '^[^a-zA-Z0-9]|[^a-zA-Z0-9]$', '')   as "Invoice No",
                    "Invoice Date" as "Invoice Date",
                     "Remarks",
                     "HL Remarks",
                    coalesce(cast(replace("Integrated Tax Amount",',','') as DECIMAL(10,2)),0) as "IGST (GL)",
                    coalesce(cast(replace("Central Tax Amount",',','') as DECIMAL(10,2)),0) as "CGST (GL)",
                    coalesce(cast(replace("State Tax Amount",',','') as DECIMAL(10,2)),0) as "SGST (GL)",
                    
                   (coalesce(cast(replace("Integrated Tax Amount",',','') as DECIMAL(10,2)),0) + 
                   coalesce(cast(replace("Central Tax Amount",',','') as DECIMAL(10,2)),0) +
                   coalesce(cast(replace("State Tax Amount",',','') as DECIMAL(10,2)),0)) 
                   as "Total Tax(GL)"
                   from
                   gl_file""")

In [17]:
gl = duckdb.sql("""
              select 
               "Supplier GSTIN/UIN",
               "MY GSTIN",
              "Invoice No",
              "Invoice Date" as"Invoice Date",
             string_agg("Remarks", '-') as "Remarks",
            string_agg("HL Remarks", '-') as "HL Remarks",
             sum("IGST (GL)") as "IGST (GL)",
             sum("CGST (GL)") as "CGST (GL)",
             sum("SGST (GL)") as "SGST (GL)",
             sum("Total Tax(GL)") as"Total Tax(GL)"
              from gl_1
              group by 1,2,3,4
              """)

In [18]:
print(f'IGST amount in gl is:-> {gl.df()["IGST (GL)"].sum()}')
print(f'CGST amount in gl is:-> {gl.df()["CGST (GL)"].sum()}')
print(f'SGST amount in gl is:-> {gl.df()["SGST (GL)"].sum()}')
print(f'Total Tax amount in gl is:-> {gl.df()["Total Tax(GL)"].sum()}')

IGST amount in gl is:-> 153907928.52999997
CGST amount in gl is:-> 117670418.69
SGST amount in gl is:-> 114199400.67
Total Tax amount in gl is:-> 385777747.89


In [19]:
g_val= gl.df()["Total Tax(GL)"].sum()
g_val

385777747.89

In [20]:
gl.shape

(88093, 10)

In [21]:
# Rank the rows in pr_file  to handle the duplicates
pr_ranked_1 = duckdb.sql("""
    SELECT "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        REGEXP_REPLACE("Document Number (PR)", '^[^a-zA-Z0-9]|[^a-zA-Z0-9]$', '') AS "Document Number (PR)",
        "Document Date (PR)",
        COALESCE(CAST(REPLACE("IGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "IGST (PR)",
        COALESCE(CAST(REPLACE("CGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "CGST (PR)",
        COALESCE(CAST(REPLACE("SGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "SGST (PR)",
        CAST(REPLACE("Total Tax (PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM pr_file
""")

pr_ranked = duckdb.sql("""
    SELECT 
       "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        "Total Tax(PR)",
        row_number() OVER (PARTITION BY "Supplier GSTIN (PR)", "My GSTIN (PR)", "Document Number (PR)", "Document Date (PR)", "Total Tax(PR)"
        order by "Document Date (PR)" asc) AS rn
    FROM pr_ranked_1
""")
# Separate the ranked data into rank 1 and duplicates
pr_filtered_1 =duckdb.sql("""
        SELECT 
        "Supplier GSTIN (PR)",
         "My GSTIN (PR)",
         "Document Number (PR)",
         "Document Date (PR)",
         COALESCE(CAST(REPLACE("IGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "IGST (PR)",
         COALESCE(CAST(REPLACE("CGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "CGST (PR)",
         COALESCE(CAST(REPLACE("SGST (PR)", ',', '') AS DECIMAL(10,2)), 0) AS "SGST (PR)",
         CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
        
    FROM pr_ranked
    where rn= 1
""")


pr_filtered =duckdb.sql("""
        SELECT 
        "Supplier GSTIN (PR)",
         "My GSTIN (PR)",
         "Document Number (PR)",
         "Document Date (PR)",
        sum("IGST (PR)") AS "IGST (PR)",
        sum("CGST (PR)")  AS "CGST (PR)",
        sum("SGST (PR)") AS "SGST (PR)",
        sum("Total Tax(PR)") AS "Total Tax(PR)"
        FROM pr_filtered_1
    group by 1,2,3,4
""")



In [22]:
pr_filtered.shape

(88218, 8)

In [23]:
pr_duplicates = duckdb.sql("""
    
    SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
            coalesce(CAST(REPLACE("IGST (PR)", ',', '') AS DECIMAL(10,2)),0) as "IGST (PR)",
             coalesce(CAST(REPLACE("CGST (PR)", ',', '') AS DECIMAL(10,2)),0) as  "CGST (PR)",
             coalesce(CAST(REPLACE("SGST (PR)", ',', '') AS DECIMAL(10,2)),0) as "SGST (PR)",
         CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
       FROM pr_ranked
    WHERE rn >= 2
""")


# Add duplicates with appropriate remarks
duplicates_df = duckdb.sql("""
    SELECT
        NULL AS "Supplier GSTIN/UIN",
        NULL AS "MY GSTIN",
        NULL AS "Invoice No", 
        NULL AS "Invoice Date", 
        NULL as "IGST (GL)",
        Null as "CGST (GL)",
        Null as "SGST (GL)",
        NULL AS "total Tax(GL)",
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)", 
        "Document Number (PR)",
        "Document Date (PR)", 
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        "Total Tax(PR)", 
        null as "Remark",
        null as "HL Remark",
        Null as "Matching keys",
       'Duplicate Entry In PR' AS "Remarks",
       Null as "Remarks_1"

    FROM pr_duplicates
""")

In [24]:
pr_duplicates.shape

(1216, 8)

In [25]:
      
 #Perform the full outer join and add remarks
result_1 = duckdb.sql("""
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)",
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered."CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        gl_file."Remarks" as "Remark" ,
        gl_file."HL Remarks" as "HL Remark" ,
        'SUPPLIER GSTIN-MY GSTIN-INVOICE NO-INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'All Match' as 'Remarks-1'
        
    FROM gl as gl_file
    FULL OUTER JOIN pr_filtered as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
    AND gl_file."Invoice Date" = pr_filtered."Document Date (PR)"
   
""")

In [26]:
# result_1 = duckdb.sql("""
#     SELECT
#         COALESCE(gl_file."Supplier GSTIN/UIN", '') AS "Supplier GSTIN/UIN",
#         COALESCE(gl_file."MY GSTIN", '') AS "MY GSTIN",
#         COALESCE(gl_file."Invoice No", '') AS "Invoice No",
#         COALESCE(gl_file."Invoice Date", '') AS "Invoice Date",
#         COALESCE(gl_file."IGST (GL)", '') AS "IGST (GL)",
#         COALESCE(gl_file."CGST (GL)", '') AS "CGST (GL)",
#         COALESCE(gl_file."SGST (GL)", '') AS "SGST (GL)",
#         COALESCE(gl_file."Total Tax(GL)", '') AS "Total Tax(GL)",
#         COALESCE(pr_filtered."Supplier GSTIN (PR)", '') AS "Supplier GSTIN (PR)",
#         COALESCE(pr_filtered."My GSTIN (PR)", '') AS "My GSTIN (PR)",
#         COALESCE(pr_filtered."Document Number (PR)", '') AS "Document Number (PR)",
#         COALESCE(pr_filtered."Document Date (PR)", '') AS "Document Date (PR)",
#         COALESCE(pr_filtered."IGST (PR)", '') AS "IGST (PR)",
#         COALESCE(pr_filtered."CGST (PR)", '') AS "CGST (PR)",
#         COALESCE(pr_filtered."SGST (PR)", '') AS "SGST (PR)",
#         COALESCE(pr_filtered."Total Tax(PR)", '') AS "Total Tax(PR)",
#         COALESCE(gl_file."Remarks", '') AS "Remark",
#         COALESCE(gl_file."HL Remarks", '') AS "HL Remark",
#         'SUPPLIER GSTIN-MY GSTIN-INVOICE NO-INVOICE DATE' AS "Matching keys",
#         CASE
#             WHEN ABS(COALESCE(gl_file."Total Tax(GL)", 0) - COALESCE(pr_filtered."Total Tax(PR)", 0)) BETWEEN 1 AND 10 THEN 'Suggested Match'
#             WHEN ABS(COALESCE(gl_file."Total Tax(GL)", 0) - COALESCE(pr_filtered."Total Tax(PR)", 0)) < 1 THEN 'Exact Match'
#             WHEN ABS(COALESCE(gl_file."Total Tax(GL)", 0) - COALESCE(pr_filtered."Total Tax(PR)", 0)) > 10 THEN 'Mismatch'
#             ELSE 'Others In Mismatch'
#         END AS "Remarks",
#         'All Match' AS "Remarks-1"
#     FROM gl AS gl_file
#     FULL OUTER JOIN pr_filtered AS pr_filtered
#     ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
#     AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
#     AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
#     AND gl_file."Invoice Date" = pr_filtered."Document Date (PR)"
# """)


In [27]:
 # AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10

In [28]:
result_1.shape

(105186, 21)

In [29]:
result_1.df().to_csv('result_1.csv', index = False)
duplicates_df.df().to_csv('Duplicates.csv', index = False)

In [30]:
matched_1 = duckdb.sql( """select * from result_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [31]:
m = matched_1.df()["Total Tax(GL)"].sum()
m

833835192.2199999

In [32]:
matched_1.df().to_csv('matched_1.csv', index = False)

In [33]:
result_1 = pd.read_csv('result_1.csv')

C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_13224\2182478160.py:1: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_1 = pd.read_csv('result_1.csv')


In [34]:
print(f'IGST amount in result_1 is:-> {result_1["IGST (GL)"].sum()},      Diff: >> {result_1["IGST (GL)"].sum() - gl.df()["IGST (GL)"].sum()})')
print(f'CGST amount in result_1  is:-> {result_1["CGST (GL)"].sum()},    Diff: >> {result_1["CGST (GL)"].sum() - gl.df()["CGST (GL)"].sum()})')
print(f'SGST amount in result_1  is:-> {result_1["SGST (GL)"].sum()},    Diff: >> {result_1["SGST (GL)"].sum() - gl.df()["SGST (GL)"].sum()}')
print(f'Total amount in result_1 is:-> {result_1["Total Tax(GL)"].sum()}, Diff: >> {result_1["Total Tax(GL)"].sum() - gl.df()["Total Tax(GL)"].sum()}')

IGST amount in result_1 is:-> 153907928.53000003,      Diff: >> 2.9802322387695312e-08)
CGST amount in result_1  is:-> 117670418.68999998,    Diff: >> 0.0)
SGST amount in result_1  is:-> 114199400.66999999,    Diff: >> 0.0
Total amount in result_1 is:-> 385777747.89, Diff: >> -5.960464477539063e-08


In [35]:
pr_2 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_1
    WHERE "Total Tax(GL)" is null""")

In [36]:
pr_2.shape

(17093, 8)

In [37]:
gl_2 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
    "total Tax(GL)",
     "Remark",
     "HL Remark"
 from result_1
 where  "Total Tax(PR)" is null """)

In [38]:
gl_2.shape

(16968, 10)

In [39]:
result_2 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)", 
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)", 
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        cast(gl_file."Remark" as string) as "Remark" ,
        cast(gl_file."HL Remark" as string) as "HL Remark",
        'SUPPLIER GSTIN, MY GSTIN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Date Not match' as "Remarks_1"
    FROM gl_2 as gl_file
    FULL OUTER JOIN pr_2 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
   
""")

In [40]:
result_2.df().to_csv('result_2.csv', index = False)

matched_2 = duckdb.sql( """select * from result_2
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

m2 =  matched_2.df()["Total Tax(GL)"].sum()

matched_2.df().to_csv('matched_2.csv', index = False)

result_2 = pd.read_csv('result_2.csv')

val = result_2["Total Tax(GL)"].sum()

print(f'total tax amount after result_2 is:-> {val+m}, Diff: >> {(val+m) - g_val})')

total tax amount after result_2 is:-> 385773272.28, Diff: >> -4475.610000014305)


C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_13224\4177071953.py:11: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_2 = pd.read_csv('result_2.csv')


In [41]:
pr_2_1 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
         "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_2
    WHERE "Supplier GSTIN/UIN" is null""")

In [42]:
gl_2_1 = duckdb.sql(""" 
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
     "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_2
 where  "Supplier GSTIN (PR)" is null """)

In [43]:
result_2_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        gl_file."Remark",
        gl_file."HL Remark",
        'SUPPLIER GSTIN, MY GSTIN, INVOICE NO(Fuzzy)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Date Not Match' as "Remarks_1"
     
    FROM gl_2_1 as gl_file
    FULL OUTER JOIN pr_2_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
    REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
     """)

In [44]:
result_2_1.df().to_csv('result_2_1.csv', index = False)

matched_2_1 = duckdb.sql( """select * from result_2_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)
m3 =  matched_2_1.df()["Total Tax(GL)"].sum() 

matched_2_1.df().to_csv('matched_2_1.csv', index = False)

result_2_1 = pd.read_csv('result_2_1.csv')

val = result_2_1["Total Tax(GL)"].sum()

print(f'IGST amount after result_2 is:-> {val+m+m2}, Diff: >> {(val+m+m2) - g_val})')

IGST amount after result_2 is:-> 385773272.2799999, Diff: >> -4475.61000007391)


C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_13224\2504443554.py:10: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_2_1 = pd.read_csv('result_2_1.csv')


In [45]:
pr_3 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_2_1
    WHERE "Supplier GSTIN/UIN" is null  """)  

In [46]:
gl_3= duckdb.sql(""" 
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark",
    "HL Remark" 
 from result_2_1
 where  "Supplier GSTIN (PR)" is null """)


In [47]:
result_3 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        gl_file."Remark" ,
        gl_file."HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, INVOICE NO, INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'MY GSTIN Not Match' as "Remarks_1"
    FROM gl_3 as gl_file
    FULL OUTER JOIN pr_3 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND  gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
        AND abs(pr_filtered."Total Tax(PR)"- gl_file."Total Tax(GL)" ) <=10
""")


In [48]:
result_3.df().to_csv('result_3.csv', index = False)

matched_3 = duckdb.sql( """select * from result_3
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)
m4 = matched_3.df()["Total Tax(GL)"].sum()

matched_3.df().to_csv('matched_3.csv', index = False)

result_3 = pd.read_csv('result_3.csv')

val = result_3["Total Tax(GL)"].sum()
print(val)
print(val+m+m2+m3)
print(m4)
print(val)

print(f'tax amount after result_2_1_1 is:-> {val+m+m2+m3}, Diff: >> {(val+m+m2+m3) - g_val})')

-470409738.68
385773272.2799999
2261521.31
-470409738.68
tax amount after result_2_1_1 is:-> 385773272.2799999, Diff: >> -4475.61000007391)


C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_13224\3755712931.py:10: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_3 = pd.read_csv('result_3.csv')


In [49]:
pr_4= duckdb.sql(""" 
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)", 
        "CGST (PR)",  
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_3
    WHERE "Supplier GSTIN/UIN" is null  """)

In [50]:
 gl_4 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark",
    "HL Remark" ,
 from result_3
 where  "Supplier GSTIN (PR)" is null """)

In [51]:
result_4 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'MY GSTIN and Date Not match' as "Remarks_1"
    FROM gl_4 as gl_file
    FULL OUTER JOIN pr_4 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
       AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
""")


In [52]:
result_4.df().to_csv('result_4.csv', index = False)

matched_4 = duckdb.sql( """select * from result_4
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)
m5 = matched_4.df()["Total Tax(GL)"].sum()

matched_4.df().to_csv('matched_4.csv', index = False)

result_4 = pd.read_csv('result_4.csv')

val = result_4["Total Tax(GL)"].sum()
print(val)
print(val+m+m2+m3+m4)
print(m4)
print(val)

print(f'tax amount after result_2_1_1 is:-> {val+m+m2+m3+m4}, Diff: >> {(val+m+m2+m3+m4) - g_val})')

-472671259.99
385773272.2799999
2261521.31
-472671259.99
tax amount after result_2_1_1 is:-> 385773272.2799999, Diff: >> -4475.61000007391)


C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_13224\1263518764.py:10: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_4 = pd.read_csv('result_4.csv')


In [53]:
pr_4_1  =duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_4
    WHERE "Supplier GSTIN/UIN" is null  """)

In [54]:
 gl_4_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
     "Remark" ,
    "HL Remark",
 from result_4
 where  "Supplier GSTIN (PR)" is null """)

In [55]:
result_4_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, INVOICE NO(FUZZY)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'MY GSTIN and Date not match' as "Remarks_1"
    FROM gl_4_1 as gl_file
    FULL OUTER JOIN pr_4_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
        REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
    AND abs(pr_filtered."Total Tax(PR)"- gl_file."Total Tax(GL)" ) <=10
      
""")


In [56]:
result_4_1.df().to_csv('result_4_1.csv', index = False)

matched_4_1 = duckdb.sql( """select * from result_4_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)
m6 = matched_4_1.df()["Total Tax(GL)"].sum()

matched_4_1.df().to_csv('matched_4_1.csv', index = False)

result_4_1 = pd.read_csv('result_4_1.csv')

val = result_4_1["Total Tax(GL)"].sum()
print(val)
print(val+m+m2+m3+m4+m5)
print(m4)
print(val)

print(f'tax amount after result_2_1_1 is:-> {val+m+m2+m3+m4+m5}, Diff: >> {(val+m+m2+m3+m4+m5) - g_val})')

-472671259.99
385773272.2799999
2261521.31
-472671259.99
tax amount after result_2_1_1 is:-> 385773272.2799999, Diff: >> -4475.61000007391)


C:\Users\rishabh.singh_clear\AppData\Local\Temp\ipykernel_13224\2162257700.py:10: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  result_4_1 = pd.read_csv('result_4_1.csv')


In [57]:
pr_5 = duckdb.sql("""
         SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_4_1
    WHERE "Supplier GSTIN/UIN" is null""")

In [58]:
 gl_5 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
     "total Tax(GL)",
     "Remark",
     "HL Remark",
 from result_4_1
 where  "Supplier GSTIN (PR)" is null """)

In [59]:
result_5 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)",
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, INVOICE NO, INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            when gl_file."Invoice No" is not null and pr_filtered."Document Number (PR)" is null then 'Missing In GSTR-1'
            when gl_file."Invoice No" is null and pr_filtered."Document Number (PR)" is not null then 'Missing In Revenue - GL'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN Not Match' as "Remarks_1"
    FROM gl_5 as gl_file
    FULL OUTER JOIN pr_5 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND  gl_file."Invoice Date" = pr_filtered."Document Date (PR)"
        AND abs(pr_filtered."Total Tax(PR)"- gl_file."Total Tax(GL)" ) <=10
        
""")

In [60]:
result_5.df().to_csv('result_5.csv', index = False)

matched_5 = duckdb.sql( """select * from result_5
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)
m7 = matched_5.df()["Total Tax(GL)"].sum()

matched_5.df().to_csv('matched_5.csv', index = False)

result_5 = pd.read_csv('result_5.csv')

val = result_5["Total Tax(GL)"].sum()
print(val)
print(val+m+m2+m3+m4+m5+m6)
print(m4)
print(val)

print(f'tax amount after result_2_1_1 is:-> {val+m+m2+m3+m4+m5+m6}, Diff: >> {(val+m+m2+m3+m4+m5+m6) - g_val})')

-472671259.99
385773272.2799999
2261521.31
-472671259.99
tax amount after result_2_1_1 is:-> 385773272.2799999, Diff: >> -4475.61000007391)


In [61]:
duplicates = pd.read_csv('Duplicates.csv')

In [62]:
final_sheet = duckdb.sql("""
            select   * from matched_1
                    union all
                    select   * from matched_2
                    union all 
                    select   * from matched_2_1
                    union all
                    select   * from matched_3 
                    union all 
                    select   * from matched_4
                    union all 
                    select   * from matched_4_1
                    union all
         
                 select * from  result_5 
                       union all 
                   select * from Duplicates.csv
""")

In [63]:
final_sheet.df().to_csv('final_sheet.csv')

In [64]:
###### BREAK after this code >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [65]:
Break break break break break break break


SyntaxError: invalid syntax (2164265595.py, line 1)

In [ ]:
 pr_6 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_5
    WHERE "Supplier GSTIN/UIN" is null """)

In [ ]:
gl_6 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
      "IGST (GL)",
      "CGST (GL)",
      "SGST (GL)",
      "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_5
 where  "Supplier GSTIN (PR)" is null """)

In [ ]:
result_6 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)",
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN and Date not match' as "Remarks_1",
        CASE
            WHEN gl_file."Total Tax(GL)" IS NULL OR pr_filtered."Total Tax(PR)" IS NULL THEN 1
            ELSE 
             row_number() OVER (PARTITION BY SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10), "MY GSTIN", "Invoice No"
                            order by "Invoice Date" ) end AS rn
    FROM gl_6 as gl_file
    FULL OUTER JOIN pr_6 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND abs(pr_filtered."Total Tax(PR)"- gl_file."Total Tax(GL)") <= 10""")


In [ ]:
matched_6 = duckdb.sql( """select * from result_6
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch')""")

In [ ]:
result_6.df().to_csv('result_6.csv', index = False)

In [ ]:
m7 = matched_6.df()["Total Tax(GL)"].sum()
matched_6.df().to_csv('matched_6.csv', index = False)

result_6 = pd.read_csv('result_6.csv')

val = result_6["Total Tax(GL)"].sum()
print(val)
print(val+m+m2+m3+m4+m5+m6)
print(m7)
print(g_val)

print(f'tax amount after result_2_1_1 is:-> {val+m+m2+m3+m4+m5+m6}, Diff: >> {(val+m+m2+m3+m4+m5+m6) - g_val})')

In [ ]:
rishabh

In [ ]:
matched_6 = duckdb.sql( """select * from result_6
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [ ]:
matched_6.df().to_csv('matched_6.csv', index= False)

In [ ]:
result_6 = pd.read_csv('result_6.csv')

In [ ]:
 pr_6_1 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_6
    WHERE "Supplier GSTIN/UIN" is null """)

In [ ]:
 gl_6_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_6
 where  "Supplier GSTIN (PR)" is null """)

In [ ]:
result_6_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)",
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
         pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, INVOICE NO(FUZZY)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Suppliers GSTIN and Date not match' as "Remarks_1"
    FROM gl_6_1 as gl_file
    FULL OUTER JOIN pr_6_1 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
        REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
""")


In [ ]:
result_6_1.df().to_csv('result_6_1.csv',index= False)

In [ ]:
matched_6_1 = duckdb.sql( """select * from result_6_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [ ]:
matched_6_1.df().to_csv('matched_6_1.csv', index= False)

In [ ]:
result_6_1 = pd.read_csv('result_6_1.csv')

In [ ]:
 pr_7= duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_6_1
    WHERE "Supplier GSTIN/UIN" is null  """)

In [ ]:
# pr_7.shape

In [ ]:
 gl_7 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
     "CGST (GL)",
     "SGST (GL)",
    "total Tax(GL)",
     "Remark",
     "HL Remark" 
 from result_6_1
 where  "Supplier GSTIN (PR)" is null """)

In [ ]:
result_7 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, INVOICE NO, INVOICE DATE' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN and MY GSTIN not match' as "Remarks_1"
    FROM gl_7 as gl_file
    FULL OUTER JOIN pr_7 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
        AND  gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
""")

In [ ]:
result_7.df().to_csv('result_7.csv',index= False)

In [ ]:
matched_7 = duckdb.sql( """select * from result_7
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [ ]:
matched_7.df().to_csv('matched_7.csv', index= False)

In [ ]:
result_7= pd.read_csv('result_7.csv')

In [ ]:
 pr_8 = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_7
    WHERE "Supplier GSTIN/UIN" is null  """)

In [ ]:
gl_8 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date", 
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
   "HL Remark" 
 from result_7
 where  "Supplier GSTIN (PR)" is null """)

In [ ]:
result_8 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, INVOICE NO' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN , MY GSTIN and Date not match' as "Remarks_1"
    FROM gl_8 as gl_file
    FULL OUTER JOIN pr_8 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND gl_file."Invoice No" = pr_filtered."Document Number (PR)"
""")


In [ ]:
result_8.df().to_csv('result_8.csv',index= False)

In [ ]:
matched_8 = duckdb.sql( """select * from result_8
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [ ]:
matched_8.df().to_csv('matched_8.csv', index= False)

In [ ]:
result_8 = pd.read_csv('result_8.csv')

In [ ]:
 pr_8_1  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_8
    WHERE "Supplier GSTIN/UIN" is null """)

In [ ]:
 gl_8_1 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_8
 where  "Supplier GSTIN (PR)" is null """)

In [ ]:
result_8_1 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, INVOICE NO(FUZZY)' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch'
        END AS "Remarks",
        'Supplier GSTIN , MY GSTIN and Date not match' as "Remarks_1"
    FROM gl_8_1 as gl_file
    FULL OUTER JOIN pr_8_1 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND REGEXP_REPLACE(gl_file."Invoice No", '[^a-zA-Z0-9]', '') =
        REGEXP_REPLACE(pr_filtered."Document Number (PR)", '[^a-zA-Z0-9]', '')
""")


In [ ]:
result_8_1.df().to_csv('result_8_1.csv',index= False)

In [ ]:
matched_8_1 = duckdb.sql( """select * from result_8_1
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """ )

In [ ]:
result_8_1 = pd.read_csv('result_8_1.csv')

In [ ]:
# only_gl_or_pr_entry = pd.read_csv('only_in_pr_gl.csv')

In [ ]:
 pr_9  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_8_1
    WHERE "Supplier GSTIN/UIN" is null 
    
    """)

In [ ]:
gl_9 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
  from result_8_1
  where  "Supplier GSTIN (PR)" is null""")

In [ ]:
result_9 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER GSTIN ,MY GSTIN and Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'Invoice Number and Date Not Match' as "Remarks_1"
    FROM gl_9 as gl_file
    FULL OUTER JOIN pr_9 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
    AND gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)"
    AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
""")


In [ ]:
result_9.df().to_csv('result_9.csv',index= False)

In [ ]:
matched_9 = duckdb.sql( """select * from result_9
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [ ]:
result_9= pd.read_csv('result_9.csv')

In [ ]:
 pr_10  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_9
    WHERE "Supplier GSTIN/UIN" is null """)

In [ ]:
 gl_10 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_9
 where  "Supplier GSTIN (PR)" is null """)

In [ ]:
result_10 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date",
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark",
        'SUPPLIER GSTIN, BUYER PAN, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'MY GSTIN , Invoice No and Date Not match' as "Remarks_1"
    FROM gl_10 as gl_file
    FULL OUTER JOIN pr_10 as pr_filtered
    ON gl_file."Supplier GSTIN/UIN" = pr_filtered."Supplier GSTIN (PR)"
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        """)


In [ ]:
result_10.df().to_csv('result_10.csv',index= False)

In [ ]:
matched_10 = duckdb.sql( """select * from result_10
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [ ]:
result_10= pd.read_csv('result_10.csv')

In [ ]:
pr_11  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_10
    WHERE "Supplier GSTIN/UIN" is null """)

In [ ]:
gl_11 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_10
 where  "Supplier GSTIN (PR)" is null """)

In [ ]:
result_11 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        case when gl_file."Remark" is null then '' else gl_file."Remark" end as "Remark" ,
        case when gl_file."HL Remark" is null then '' else gl_file."HL Remark" end as "HL Remark" ,
        'SUPPLIER PAN, MY GSTIN, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'SupplierGSTIN , Invoice No and Date Not match' as "Remarks_1"
    FROM gl_11 as gl_file
    FULL OUTER JOIN pr_11 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND gl_file."MY GSTIN"  = pr_filtered."My GSTIN (PR)"
        AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        
""")

   

In [ ]:
result_11.df().to_csv('result_11.csv',index= False)

In [ ]:
matched_11 = duckdb.sql( """select * from result_11
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [ ]:
result_11= pd.read_csv('result_11.csv')

In [ ]:
pr_12  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_11
    WHERE "Supplier GSTIN/UIN" is null """)

In [ ]:
gl_12 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_11
 where  "Supplier GSTIN (PR)" is null """)

In [ ]:
result_12 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
        'SUPPLIER PAN, BUYER PAN, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'Supplier GSTIN , MY GSTIN , Invoice No and Date not match' as "Remarks_1"
    FROM gl_12 as gl_file
    FULL OUTER JOIN pr_12 as pr_filtered
    ON SUBSTRING(gl_file."Supplier GSTIN/UIN", 3, 10)  = SUBSTRING(pr_filtered."Supplier GSTIN (PR)", 3, 10)
        AND SUBSTRING(gl_file."MY GSTIN", 3, 10)  = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10) 
        AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        
""")

   

In [ ]:
result_12.df().to_csv('result_12.csv',index= False)

In [ ]:
matched_12 = duckdb.sql( """select * from result_12
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [ ]:
result_12 = pd.read_csv('result_12.csv')

In [ ]:
pr_13  = duckdb.sql("""
 SELECT 
        "Supplier GSTIN (PR)",
        "My GSTIN (PR)",
        "Document Number (PR)",
        "Document Date (PR)",
        "IGST (PR)",
        "CGST (PR)",
        "SGST (PR)",
        CAST(REPLACE("Total Tax(PR)", ',', '') AS DECIMAL(10,2)) AS "Total Tax(PR)"
    FROM result_12
    WHERE "Supplier GSTIN/UIN" is null """)

In [ ]:
gl_13 = duckdb.sql("""
SELECT 
    "Supplier GSTIN/UIN",
     "MY GSTIN",
     "Invoice No", 
     "Invoice Date",
     "IGST (GL)",
    "CGST (GL)",
    "SGST (GL)",
    "total Tax(GL)",
    "Remark" ,
    "HL Remark"
 from result_11
 where  "Supplier GSTIN (PR)" is null """)

In [ ]:
result_13 = duckdb.sql( """
        SELECT
        gl_file."Supplier GSTIN/UIN",
        gl_file."MY GSTIN" ,
        gl_file."Invoice No", 
        gl_file."Invoice Date", 
        gl_file."IGST (GL)",
        gl_file."CGST (GL)",
        gl_file."SGST (GL)",
        gl_file."Total Tax(GL)" ,
        pr_filtered."Supplier GSTIN (PR)", 
        pr_filtered."My GSTIN (PR)" ,
        pr_filtered."Document Number (PR)",
        pr_filtered."Document Date (PR)" ,
        pr_filtered."IGST (PR)",
        pr_filtered. "CGST (PR)",
        pr_filtered."SGST (PR)",
        pr_filtered."Total Tax(PR)",
        "Remark" ,
        "HL Remark" ,
         'BUYER GSTIN/PAN, INOVICE NO, Tax Amount' as "Matching keys",
        CASE
            WHEN ABS(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") BETWEEN 1 AND 10 THEN 'Suggested Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") <1  THEN 'Exact Match'
            WHEN abs(gl_file."Total Tax(GL)" - pr_filtered."Total Tax(PR)") >10  THEN 'Mismatch'
            ELSE 'Others In Mismatch Or Entry only in PR/GL'
        END AS "Remarks",
        'Supplier GSTIN(PAN),MY GSTIN , Invoice No and Date not match' as "Remarks_1"
    FROM gl_13 as gl_file
    FULL OUTER JOIN pr_13 as pr_filtered
    ON 
     (
            gl_file."MY GSTIN" = pr_filtered."My GSTIN (PR)" 
            OR SUBSTRING(gl_file."MY GSTIN", 3, 10) = SUBSTRING(pr_filtered."My GSTIN (PR)", 3, 10)
        )
AND gl_file."Invoice Date" = pr_filtered."Document Date (PR)" 
AND abs(pr_filtered."Total Tax(PR)"-gl_file."Total Tax(GL)" ) <=10
        
""")

   

In [ ]:
result_13.df().to_csv('result_13.csv',index= False)

In [ ]:
matched_13 = duckdb.sql( """select * from result_13
                               
                               where Remarks in ('Exact Match', 'Suggested Match', 'Mismatch') """)

In [ ]:
matched_13.shape

In [ ]:
combined_sheet = duckdb.sql("""
                    select   * from matched_1
                    union all
                    select   * from matched_2
                    union all 
                    select   * from matched_2_1
                    union all
                    select   * from matched_3 
                    union all 
                    select   * from matched_4
                    union all 
                    select   * from matched_4_1
                    union all
                    select * from matched_5
                    union all
                    select * from matched_6
                    union all 
                    select * from matched_6_1
                    union all 
                    select * from matched_7
                    union all
                    select * from matched_8
                    union all 
                    select * from matched_8_1
                    union all
                    select * from matched_9
                    union all
                    select * from matched_10
                    union all
                    select * from matched_11
                    union all
                    select * from matched_12
                    union all
                    select * from result_13
                    
                     """)


In [ ]:
combined_sheet.df().to_csv('combined_sheet.csv', index = False)

In [ ]:
duplicates_df = pd.read_csv("Duplicates.csv")

In [ ]:
final_sheet = duckdb.sql("""
    select * from combined_sheet
    union all
    select * from duplicates_df
""")

In [ ]:
final_sheet.df().to_csv('final_sheet.csv', index= False)